In [1]:
# Import our dependencies
import warnings\

warnings.filterwarnings("ignore")
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("resources/charity_data.csv")
application_df.head()

,EIN,NAME,APPLICATION_TYPE,AFFILIATION,CLASSIFICATION,USE_CASE,ORGANIZATION,STATUS,INCOME_AMT,SPECIAL_CONSIDERATIONS,ASK_AMT,IS_SUCCESSFUL
0,10520599,BLUE KNIGHTS MOTORCYCLE CLUB,T10,Independent,C1000,ProductDev,Association,1,0,N,5000,1
1,10531628,AMERICAN CHESAPEAKE CLUB CHARITABLE TR,T3,Independent,C2000,Preservation,Co-operative,1,1-9999,N,108590,1
2,10547893,ST CLOUD PROFESSIONAL FIREFIGHTERS,T5,CompanySponsored,C3000,ProductDev,Association,1,0,N,5000,0
3,10553066,SOUTHSIDE ATHLETIC ASSOCIATION,T3,CompanySponsored,C2000,Preservation,Trust,1,10000-24999,N,6692,1
4,10556103,GENETIC RESEARCH INSTITUTE OF THE DESERT,T3,Independent,C1000,Heathcare,Trust,1,100000-499999,N,142590,1


In [2]:
#Drop unneccessary columns
application_df = application_df.drop(['EIN', 'NAME'], axis=1)

In [3]:
#Get values
app_count = application_df['APPLICATION_TYPE'].value_counts()

In [4]:
list_index = [i for i, x in enumerate(app_count) if x < 750]
application_types_to_replace = [app_count.index[x] for x in list_index]

for app in application_types_to_replace:
    application_df['APPLICATION_TYPE'] = application_df['APPLICATION_TYPE'].replace(app,"Other")
# Check to make sure binning was successful
application_df['APPLICATION_TYPE'].value_counts()

T3       27037
Other     2266
T4        1542
T6        1216
T5        1173
T19       1065
Name: APPLICATION_TYPE, dtype: int64

In [5]:
class_count = application_df['CLASSIFICATION'].value_counts()

list_index_class = [i for i, x in enumerate(class_count) if x < 1900]
classifications_to_replace = [class_count.index[x] for x in list_index_class]

# Replace in dataframe
for cls in classifications_to_replace:
    application_df['CLASSIFICATION'] = application_df['CLASSIFICATION'].replace(cls,"Other")
    
# Check to make sure binning was successful
application_df['CLASSIFICATION'].value_counts()

C1000    17326
C2000     6074
C1200     4837
Other     4144
C3000     1918
Name: CLASSIFICATION, dtype: int64

In [6]:
application_dummies = pd.get_dummies(application_df)

In [7]:
# Split our preprocessed data into our features and target arrays
y = application_dummies['IS_SUCCESSFUL'].values
X = application_dummies.drop(['IS_SUCCESSFUL'], axis = 1)

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 42)

In [8]:
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [9]:
dim= len(X_train_scaled[0])
print(dim)

39


In [10]:
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=dim*2, activation="relu", input_dim=dim))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=dim/2, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 78)                3120      
_________________________________________________________________
dense_1 (Dense)              (None, 19)                1501      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 20        
Total params: 4,641
Trainable params: 4,641
Non-trainable params: 0
_________________________________________________________________


In [11]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [12]:
#Running model with more frequent app/classifcation
mc = tf.keras.callbacks.ModelCheckpoint('training-opt1/weights{epoch:04d}.h5', 
                                     save_weights_only=True, save_freq=5)
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[mc])

Epoch 1/50
25724/25724 [==============================] - 10s 383us/sample - loss: 0.5783 - acc: 0.7178
Epoch 2/50
25724/25724 [==============================] - 10s 381us/sample - loss: 0.5627 - acc: 0.7254
Epoch 3/50
25724/25724 [==============================] - 10s 379us/sample - loss: 0.5606 - acc: 0.7270
Epoch 4/50
25724/25724 [==============================] - 9s 357us/sample - loss: 0.5588 - acc: 0.7262
Epoch 5/50
25724/25724 [==============================] - 9s 344us/sample - loss: 0.5569 - acc: 0.7280
Epoch 6/50
25724/25724 [==============================] - 8s 330us/sample - loss: 0.5572 - acc: 0.7281
Epoch 7/50
25724/25724 [==============================] - 8s 327us/sample - loss: 0.5557 - acc: 0.7291
Epoch 8/50
25724/25724 [==============================] - 8s 326us/sample - loss: 0.5545 - acc: 0.7298
Epoch 9/50
25724/25724 [==============================] - 8s 325us/sample - loss: 0.5534 - acc: 0.7294
Epoch 10/50
25724/25724 [==============================] - 8s 328us/sa

In [13]:
#Added fourth layer for optimization
nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=dim*2, activation="relu", input_dim=dim))
# Second hidden layer
nn.add(tf.keras.layers.Dense(units=dim/2, activation="relu"))
# Added third layer
nn.add(tf.keras.layers.Dense(units=dim/4, activation="relu"))
# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 78)                3120      
_________________________________________________________________
dense_4 (Dense)              (None, 19)                1501      
_________________________________________________________________
dense_5 (Dense)              (None, 9)                 180       
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 10        
Total params: 4,811
Trainable params: 4,811
Non-trainable params: 0
_________________________________________________________________


In [15]:
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

mc2 = tf.keras.callbacks.ModelCheckpoint('training-opt2/weights{epoch:04d}.h5', 
                                     save_weights_only=True, save_freq=5)
fit_model = nn.fit(X_train_scaled, y_train, epochs=50, callbacks=[mc2])

Epoch 1/50
25724/25724 [==============================] - 10s 372us/sample - loss: 0.5796 - acc: 0.7158 - loss: 0.5806 - acc: 0.
Epoch 2/50
25724/25724 [==============================] - 9s 367us/sample - loss: 0.5621 - acc: 0.7255
Epoch 3/50
25724/25724 [==============================] - 9s 367us/sample - loss: 0.5589 - acc: 0.7284
Epoch 4/50
25724/25724 [==============================] - 10s 370us/sample - loss: 0.5575 - acc: 0.7287
Epoch 5/50
25724/25724 [==============================] - 10s 373us/sample - loss: 0.5559 - acc: 0.7301
Epoch 6/50
25724/25724 [==============================] - 10s 371us/sample - loss: 0.5555 - acc: 0.7303
Epoch 7/50
25724/25724 [==============================] - 10s 405us/sample - loss: 0.5545 - acc: 0.7305
Epoch 8/50
25724/25724 [==============================] - 10s 370us/sample - loss: 0.5540 - acc: 0.7310
Epoch 9/50
25724/25724 [==============================] - 10s 391us/sample - loss: 0.5534 - acc: 0.7313
Epoch 10/50
25724/25724 [================

In [16]:
#Trying to get keras to find the best parameters for us
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=1,
        max_value=78,
        step=13), activation=activation, input_dim=39))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 6)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=1,
            max_value=60,
            step=5),
            activation=activation))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["acc"])
    
    return nn_model

In [17]:
import kerastuner as kt
#create create tuner object to iterate through possibilities
tuner = kt.Hyperband(
    create_model,
    objective="val_acc",
    max_epochs=20,
    hyperband_iterations=2)

In [18]:
#Search through params for best case scenario
tuner.search(X_train_scaled, y_train,epochs=20, validation_data=(X_test_scaled,y_test))

Trial 60 Complete [00h 00m 18s]
val_acc: 0.7302623987197876

Best val_acc So Far: 0.7308454513549805
Total elapsed time: 00h 07m 49s
INFO:tensorflow:Oracle triggered exit


In [19]:
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'activation': 'relu',
 'first_units': 53,
 'num_layers': 2,
 'units_0': 31,
 'units_1': 51,
 'units_2': 36,
 'units_3': 26,
 'units_4': 31,
 'units_5': 36,
 'tuner/epochs': 7,
 'tuner/initial_epoch': 0,
 'tuner/bracket': 1,
 'tuner/round': 0}

In [21]:
#unfortunately the best model isn't better
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

8575/8575 - 0s - loss: 0.5589 - acc: 0.7308
Loss: 0.5589260859218352, Accuracy: 0.7308454513549805
